In [1]:
import pandas as pd
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT
from sqlalchemy import text
from sqlalchemy import create_engine


In [2]:
# connect to the database
engine = create_engine('mssql+pyodbc://WagnerProdAGL1/Targit_DM?driver=ODBC+Driver+17+for+SQL+Server', echo=False, fast_executemany=True)

In [8]:
# Read OTIL excel
df_OTIL_Stage = pd.read_excel('Files\CCPA Line Detail Wagner December 2023.xlsx', sheet_name='CCPA Line Detail')

In [9]:
df_OTIL_Stage.head(5)

,Main Dealer,Frcst Enty Dlr Cd,Dlr Cd Emer,Main Dlr Cd,Dealer Cd,Sales Channel,Part Number,Sourced Part,TEPS Ind,CES Ind,...,End Cust Ord No,Order Entry User Id,Stocking Ind,End Cust Ln No,Src Fac,Str Nm,Key Cat Account Num,Frcst Ind,Days To Fill_SP,Line Cost Center
0,E250,E250,E250,E250,E250 - WAGNER EQ,WO,2397352,2397352,N,N,...,AGS155049,XUPE25TDG,Y,12,0,AURORA,CEB2,Y,-2,PD
1,E250,E250,E250,E250,E250 - WAGNER EQ,WO,2933645,2933645,N,N,...,AGS157417,E250JRS,Y,4,0,AURORA,CCHD,Y,-1,CA
2,E250,E250,E250,E250,E250 - WAGNER EQ,WO,1950936,1950936,N,N,...,AGS155983,E250JH5,Y,1,0,AURORA,NaN,Y,-1,MS
3,E250,E250,E250,E250,E250 - WAGNER EQ,OTC,8D7424,8D7424,N,N,...,AGC585927,XUPE25EZA,Y,6,0,AURORA,NaN,Y,-1,DN
4,E250,E250,E350,E250,E350 - WAGNER EQ,PS,5280585,5280585,N,N,...,1AC736938,treeves,Y,2,10,ALBUQUERQUE,NaN,Y,-1,PD


### Logic to getting the calculations made

In [10]:
# Create df_OTIL with required columns
df_OTIL = df_OTIL_Stage[['Dlr Cd Emer', 'OTIL LINES']].copy()

# Calculate count of '1' in column 'OTIL LINES' for each dealer code
df_count_ones = df_OTIL[df_OTIL['OTIL LINES'] == 1].groupby('Dlr Cd Emer')['OTIL LINES'].count().reset_index()
df_count_ones = df_count_ones.rename(columns={'OTIL LINES': 'Y'})

# Calculate total count in column 'OTIL LINES' for each dealer code
df_total_count = df_OTIL.groupby('Dlr Cd Emer')['OTIL LINES'].count().reset_index()
df_total_count = df_total_count.rename(columns={'OTIL LINES': 'Grand Total'})

# Merge the two dataframes on 'Dlr Cd Emer'
df_result = pd.merge(df_count_ones, df_total_count, on='Dlr Cd Emer', how='right')

# Fill NaN values with 0 in case there are no '1' occurrences for a dealer code
df_result = df_result.fillna(0)

# Calculate the Percentage column
df_result['Percentage'] = (df_result['Y'] / df_result['Grand Total']) * 100

# Convert 'Dlr Cd Emer' to string explicitly
df_result['Dlr Cd Emer'] = df_result['Dlr Cd Emer'].astype(str)

In [11]:
df_result.head(5)

,Dlr Cd Emer,Y,Grand Total,Percentage
0,E250,39517,40590,97.356492
1,E251,2868,2907,98.658411
2,E252,17994,18089,99.474819
3,E253,7727,8053,95.951819
4,E254,2623,2710,96.789668


In [12]:
# Specify data types for each column
dtype_mapping = {
    'Dlr Cd Emer': VARCHAR,
    'Y': INTEGER,
    'Grand Total': INTEGER,
    'Percentage': FLOAT,
}


In [13]:
# Convert the DataFrame to a table in the dbo schema without explicit data types
df_result.to_sql('OTIL_Stage', con=engine, index=False, if_exists='replace', schema='dbo')

-1

In [14]:
# Define the SQL query to create the final table
sql_create_table_query = """
CREATE TABLE [Targit_DM].[dbo].[OTIL_data] (
    [Dealer_Code] VARCHAR(25),
    [Rollup_Store] VARCHAR(10),
    [Y] INT,
    [Grand_Total] INT,
    [Percentage] FLOAT
)
"""

In [15]:
# Define the SQL query to insert into the final table
sql_insert_query = """
INSERT INTO [Targit_DM].[dbo].[OTIL_data]
SELECT 
       t1.[Dlr Cd Emer]
      ,t2.[Rollup_Store]
      ,t1.[Y]
      ,t1.[Grand Total]
      ,t1.[Percentage]
FROM [dbo].[OTIL_Stage] t1
LEFT JOIN [Daisy\Wagner].[XREF].[dbo].[Parts_KPIDealerCodeStoreRollup] t2
ON t1.[Dlr Cd Emer] = t2.Dealer_Code
"""

In [16]:
# Define the SQL query to truncate the staging table
sql_truncate_staging = "TRUNCATE TABLE [Targit_DM].[dbo].[OTIL_Stage]"

In [17]:
# Execute the SQL queries
with engine.connect() as connection:
    # Create the final table
    connection.execute(text(sql_create_table_query))
    
    # Insert into final table
    connection.execute(text(sql_insert_query))
    
    # Truncate staging table
    connection.execute(text(sql_truncate_staging))

    # Commit the transaction
    connection.commit()


In [ ]:
# Write df_OTIL to SQL Server (replace 'final_table' with your actual table name)
# df_OTIL.to_sql('Parts_OTIL', con=engine, index=False, if_exists='replace')


# Write df_OTIL to Excel
df_OTIL.to_excel('OTIL_Testing.xlsx', index=False)